In [1]:
import math
import numpy as np
import pandas as pd
import os
from collections import Counter
from random import shuffle
from tqdm import tqdm
import cv2
TRAIN_DIR='G:\FYPDATA2'
IMG_SIZE=80

In [2]:
##need to filter successful grasps only. 
def split_to_index(img_label):
    #splits grasp_0_0_colour.jpg to just the '0' in int
    return int(img_label.split('.')[0].split('_')[1])

In [3]:
def create_train_data():
    training_data = []
    label_data=[]
    img_data=[]
    n=0
    for folder in tqdm(os.listdir(TRAIN_DIR)):
        folder_increment=2000*n
        n=n+1 #each time it's a new folder. Increment by 2000
        for file in os.listdir(TRAIN_DIR+'/'+folder):
            if file.endswith(".csv"):
                index=split_to_index(file)+folder_increment
                grasp_csv=pd.read_csv(TRAIN_DIR+'/'+folder+'/'+file)
                angle=grasp_csv['Angle'][1]
                success=grasp_csv['success'][1]
                if angle>=-180 and angle<0:
                    #180 of the gripper is the same. avoids confusion in CNN.
                    angle=angle+180 
                label=[0,0,0,0,0,0]
                dense_angle=math.floor(angle/30) #from 1-6 (0-180)
                label[dense_angle]=1
                label_data.append([np.array(label),success,index])

            # grab first image of every grasp
            if file.endswith("0_colour.jpg"):
                index=split_to_index(file)+folder_increment
                path = os.path.join(TRAIN_DIR+'/'+folder,file) ##path of the image TRAIN_DIR+img
                img = cv2.imread(path,cv2.IMREAD_GRAYSCALE) ##read img in grayscale
                img = cv2.resize(img, (IMG_SIZE,IMG_SIZE)) ##resize the img to 50,50
                img_data.append([img,index])
    return label_data, img_data

In [4]:
[label_data,img_data]=create_train_data()

100%|██████████████████████████████████████████| 11/11 [01:39<00:00, 10.32s/it]


In [34]:
cv2.imshow('iameg',img_data[0][0])
cv2.waitKey()

-1

In [14]:
img_data_df=pd.DataFrame(img_data,columns=['image','id'])
img_data_df.head()
#len(label_data_df.id.unique())

,image,id
0,"[[254, 254, 254, 254, 254, 254, 254, 254, 254,...",0
1,"[[254, 254, 254, 254, 254, 254, 254, 254, 254,...",100
2,"[[254, 254, 254, 254, 254, 254, 254, 254, 254,...",101
3,"[[254, 254, 254, 254, 254, 254, 254, 254, 254,...",102
4,"[[254, 254, 254, 254, 254, 254, 254, 254, 254,...",103


In [15]:
label_data_df=pd.DataFrame(label_data,columns=['labels','success','id'])
label_data_df.head()
#len(label_data_df.id.unique())

,labels,success,id
0,"[0, 0, 0, 0, 1, 0]",1,0
1,"[0, 0, 0, 1, 0, 0]",1,1
2,"[0, 0, 0, 1, 0, 0]",1,10
3,"[0, 1, 0, 0, 0, 0]",1,100
4,"[1, 0, 0, 0, 0, 0]",0,101


In [23]:
final_data_df=img_data_df.merge(label_data_df,how='inner',on='id')
#where the success is 1
final_data_df=final_data_df[final_data_df['success']==1]
final_data_df.head()
#len(final_data_df.id.unique())

,image,id,labels,success
0,"[[254, 254, 254, 254, 254, 254, 254, 254, 254,...",0,"[0, 0, 0, 0, 1, 0]",1
1,"[[254, 254, 254, 254, 254, 254, 254, 254, 254,...",100,"[0, 1, 0, 0, 0, 0]",1
3,"[[254, 254, 254, 254, 254, 254, 254, 254, 254,...",102,"[0, 0, 0, 0, 1, 0]",1
5,"[[254, 254, 254, 254, 254, 254, 254, 254, 254,...",104,"[0, 0, 0, 1, 0, 0]",1
6,"[[254, 254, 254, 254, 254, 254, 254, 254, 254,...",105,"[0, 0, 0, 0, 0, 1]",1


In [24]:
final_data=final_data_df.values


In [25]:
shuffle(final_data)

In [26]:
np.save('training_data_v1.npy', final_data)

In [14]:
# label=[0,0,0,0,0,0,0,0,0,0,0,0]
# dense_angle=int(round(-160/30))+6 #from 1-11 (-165 to 165)
# label[dense_angle]=1
# label

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

5